## First we initialize our globals

In [42]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from databricks.connect import DatabricksSession
spark = DatabricksSession.builder.getOrCreate()

In [44]:
# add project dir to python path
try:
    notebook_path = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
    import sys, os
    sys.path.append("/Workspace" + os.path.dirname(os.path.dirname(notebook_path)))
except:
    pass

In [45]:
# load table into spark dataframe
cows_bff = spark.read.table("farm.db.cows_bff")

## Some graphs and charts

In [46]:
display(cows_bff.limit(10))

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:567: FutureWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:386: FutureWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.



,cow_name,meal_start,meal_end,meal_start_time,meal_end_time,duration,date,day
0,Butterscotch,32500,33618,9:01:40,9:20:18,1118,2023-05-20,Saturday
1,Hershey,32205,33303,8:56:45,9:15:03,1098,2023-05-20,Saturday
2,Mocha,33182,33264,9:13:02,9:14:24,82,2023-05-20,Saturday
3,Cocoa,33182,33264,9:13:02,9:14:24,82,2023-05-20,Saturday
4,Nutella,32587,33021,9:03:07,9:10:21,434,2023-05-20,Saturday
5,Brandy,33847,34406,9:24:07,9:33:26,559,2023-05-20,Saturday
6,Peaches,32122,32158,8:55:22,8:55:58,36,2023-05-20,Saturday
7,Marshmallow,34292,34530,9:31:32,9:35:30,238,2023-05-20,Saturday
8,Popcorn,31979,34004,8:52:59,9:26:44,2025,2023-05-20,Saturday
9,Muffin,32396,34507,8:59:56,9:35:07,2111,2023-05-20,Saturday


## Lets compute meal overlap

In [47]:
from cow_bff.heatmap import compute_heatmap, compute_overlap

overlap = compute_overlap(cows_bff)

df = compute_heatmap(overlap)

display(df.limit(10))

,cow1,cow2,distance
0,Brandy,Buttercup,603.076923
1,Brandy,Butterscotch,395.384615
2,Brandy,Cocoa,323.307692
3,Brandy,Daisy,528.230769
4,Brandy,Dottie,447.615385
5,Brandy,Hershey,588.692308
6,Brandy,Magic,620.384615
7,Brandy,Marshmallow,370.076923
8,Brandy,Mocha,323.307692
9,Brandy,Muffin,425.307692


# Display Heatmap

In [48]:
pdf = df.toPandas()
pdf = pdf.pivot(index='cow1', columns='cow2', values='distance').fillna(0)

import plotly.express as px
fig = px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"), title="Cow BFFs", color_continuous_scale='redor')
px.imshow(pdf, x=pdf.columns, y=pdf.index, labels=dict(x="Cow 2", y="Cow 1", color="Distance"),)

fig.update_layout(width=800,height=500)
fig.show()


In [59]:
from cow_bff.historical import identify_friend_groups

friend_groups = identify_friend_groups(n=3, distance=df, overlap=overlap)

+------+-------+-----------------+-----------------+-------------------+-------+
|  cow1|   cow2|         distance|     friend_group|              date1|overlap|
+------+-------+-----------------+-----------------+-------------------+-------+
| Mocha|  Cocoa|878.6923076923077|   [Mocha, Cocoa]|2023-05-20 00:00:00|     82|
| Cocoa|  Mocha|878.6923076923077|   [Cocoa, Mocha]|2023-05-20 00:00:00|     82|
|Brandy|Nutella|634.3076923076923|[Brandy, Nutella]|2023-05-20 00:00:00|      0|
| Mocha|  Cocoa|878.6923076923077|   [Mocha, Cocoa]|2023-05-21 00:00:00|    251|
| Cocoa|  Mocha|878.6923076923077|   [Cocoa, Mocha]|2023-05-21 00:00:00|    251|
|Brandy|Nutella|634.3076923076923|[Brandy, Nutella]|2023-05-21 00:00:00|    951|
| Mocha|  Cocoa|878.6923076923077|   [Mocha, Cocoa]|2023-05-22 00:00:00|    174|
| Cocoa|  Mocha|878.6923076923077|   [Cocoa, Mocha]|2023-05-22 00:00:00|    174|
|Brandy|Nutella|634.3076923076923|[Brandy, Nutella]|2023-05-22 00:00:00|   1192|
| Mocha|  Cocoa|878.69230769

In [57]:
besties_tracking = cows_bff.join(friend_groups, on=[cows_bff.cow_name == friend_groups.cow1, cows_bff.date == friend_groups.date1]) \
    .select("friend_group", "duration", "date", "day")


In [58]:
bestiespdf = besties_tracking.toPandas().astype({'friend_group': 'string'})
import plotly.express as px

fig = px.line(bestiespdf, x='date', y = 'duration', color='friend_group')
fig.show()

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:567: FutureWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/pyspark/sql/pandas/types.py:386: FutureWarning:

is_datetime64tz_dtype is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.DatetimeTZDtype)` instead.

/Users/kimberly.mahoney/.pyenv/versions/3.10.12/envs/cows-bff/lib/python3.10/site-packages/_plotly_utils/basevalidators.py:105: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result

